In [ ]:
# users : 200000 
# musics : 127771

import music_rating

(x_train, y_train), (x_test, y_test) = music_rating.Data().load_data()
user_train, user_test = x_train[0], x_test[0]
song_train, song_test = x_train[1], x_test[1]

from keras.layers import Embedding, Dense, Input, Reshape, Concatenate
from keras.models import Model

n_embedding = 256 # 5
user_set = set(user_train.flatten().tolist() + user_test.flatten().tolist())
song_set = set(song_train.flatten().tolist() + song_test.flatten().tolist())
n_input_user = len(user_set)
n_input_song = len(song_set)

user_id_input = Input(shape=[1], name='user')
song_id_input = Input(shape=[1], name='song')

user_embedding = Embedding(output_dim=n_embedding, input_dim=n_input_user,
                           input_length=1, name='user_embedding')(user_id_input)
song_embedding = Embedding(output_dim=n_embedding, input_dim=n_input_song,
                           input_length=1, name='song_embedding')(song_id_input)

user_vecs = Reshape([n_embedding])(user_embedding)
song_vecs = Reshape([n_embedding])(song_embedding)
input_vecs = Concatenate()([user_vecs, song_vecs])

x = Dense(128, activation='sigmoid')(input_vecs)
#x = Dense(64, activation='sigmoid')(x)

y = Dense(1)(x)

model = Model(inputs=[user_id_input, song_id_input], outputs=y)
#model.compile(optimizer='adam', loss='mse')
model.compile(optimizer='adagrad', loss='mse')

model.fit([user_train, song_train], y_train, epochs=20, batch_size = 500, verbose=1)
loss = model.evaluate([user_test, song_test], y_test)
print('mse loss = ', loss)